In [1]:
import boto3

In [6]:
def iterate_bucket(bucket):
    client = boto3.client('s3')
    paginator = client.get_paginator('list_objects_v2')
    page_iterator = paginator.paginate(Bucket=bucket)
    
    for page in page_iterator:
        if page['KeyCount'] > 0:
            for item in page['Contents']:
                yield item

In [19]:
counter=0
pathlist=[]
for i in iterate_bucket(bucket='millionsongproject'):
    if counter<200000:
        if i['Key'].endswith('.h5'):
            pathlist.append(i['Key'])
            counter+=1
    else:
        break

In [21]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import SparkSession
sc = SparkContext()
ss=SparkSession \
.builder \
.getOrCreate()

In [22]:
path_rdd=sc.parallelize(pathlist)

In [25]:
path_rdd.take(5)

['A/A/A/TRAAAAK128F9318786.h5',
 'A/A/A/TRAAAAV128F421A322.h5',
 'A/A/A/TRAAAAW128F429D538.h5',
 'A/A/A/TRAAAAY128F42A73F0.h5',
 'A/A/A/TRAAABD128F429CF47.h5']

In [32]:
import hdf5_getters
import pandas as pd
import numpy as np

In [33]:
# get all getters
onegetter = ''
getters = filter(lambda x: x[:4] == 'get_', hdf5_getters.__dict__.keys())
getters = list(getters)
getters.remove("get_num_songs") # special case
if onegetter == 'num_songs' or onegetter == 'get_num_songs':
    getters = []
elif onegetter != '':
    if onegetter[:4] != 'get_':
        onegetter = 'get_' + onegetter
    try:
        getters.index(onegetter)
    except ValueError:
        print('ERROR: getter requested:',onegetter,'does not exist.')
        sys.exit(0)
    getters = [onegetter]
getters = np.sort(getters)
attribute_list = []
#creating the dataframe
for getter in getters:
    attribute_list.append(getter[4:])
print('no. of attributes = ',len(attribute_list))
subset_df = pd.DataFrame(columns=attribute_list)

no. of attributes =  54


In [41]:
for i in path_rdd.take(10):
    address = 's3://millionsongproject/'+i
    h5 = pd.read_hdf(address)
    numSongs = hdf5_getters.get_num_songs(h5)
    # print them
    for getter in getters:
        try:
            res = hdf5_getters.__getattribute__(getter)(h5,songidx)
        except AttributeError:
            if summary:
                continue
            else:
                #print(e)
                print('forgot -summary flag? specified wrong getter?')
        if res.__class__.__name__ == 'ndarray':
            #print(getter[4:]+": shape =",res.shape)
            subset_df.loc[i,getter[4:]] = res.shape
        else:
            #print(getter[4:]+":",res)
            subset_df.loc[i,getter[4:]] = res

    # done
    #print('DONE, showed song',songidx,'/',numSongs-1,'in file:',hdf5path)
    h5.close()

FileNotFoundError: File s3://millionsongproject/A/A/A/TRAAAAK128F9318786.h5 does not exist